# ChatGPT exported conversation search and rendering tool

## Initialization

### Imports

In [1]:
import json
import re
from configparser import ConfigParser
from datetime import datetime
from IPython.display import display, Markdown
import ipywidgets as widgets

### Function definitions

In [2]:
returns = {}

def conversation_search(search_str, conversations, search_only_from_title=True, case_sensitive=False):
    results = []
    for i, conversation in enumerate(conversations):
        result = {}
        append_flag = False
        title = conversation.get("title")
        conversation_str = return_conversation_string(conversation)
        if case_sensitive and search_only_from_title and search_str in title:
            append_flag = True
        elif case_sensitive and not search_only_from_title and (search_str in title or search_str in conversation_str):
            append_flag = True
        elif not case_sensitive and search_only_from_title and search_str.lower() in title.lower():
            append_flag = True
        elif not case_sensitive and not search_only_from_title and (search_str.lower() in title.lower() or search_str.lower() in conversation_str.lower()):
            append_flag = True
        if append_flag:
            timestamp = conversation.get("create_time")
            iso_time = datetime.fromtimestamp(timestamp).strftime("%F %T")
            result.update(
                {
                    "list_index": i,
                    "id": conversation.get("conversation_id"),
                    "creation_iso_time": iso_time,
                    "title": title,
                }
            )
            results.append(result)
    returns["results"] = results
    global conv_selection
    conv_selection.options = [f"""{result.get("creation_iso_time")} {result.get("title").strip()}""" for result in results]
    conv_selection.layout.height = f"{max([24, (len(conv_selection.options) - 1)*16.61 + 24])}px"
    return results

def return_conversation_string(conversation):
    messages = []
    for key, value in conversation.get("mapping").items():
        message = value.get("message")
        if message and not message.get("is_visually_hidden_from_conversation"):
            parts = message.get("content").get("parts")
            if parts and message.get("content").get("content_type") == "text":
                assert len(parts) == 1
                messages.append(parts[0])
    message_str = "\n".join(messages)
    return message_str


def conv_to_markdown(conversation):
    markdown = ""
    create_time = conversation.get("create_time")
    title = conversation.get("title")
    conversation_id = conversation.get("conversation_id")
    if create_time and title and conversation_id:
        title = title.strip()
        iso_time = datetime.fromtimestamp(create_time).strftime("%F %T")
        url_prefix = "https://chatgpt.com/c/"
        url = url_prefix + conversation_id
        markdown += f"# <u>{title}</u>\n"
        markdown += f"**{iso_time} - [{url}]({url})**  \n\n"
        markdown += "---\n\n"
    for mapping in conversation.get("mapping").values():
        message = mapping.get("message")
        if not message:
            continue
        content = message.get("content")
        if not content:
            continue
        if content.get("content_type") != "text":
            continue
        if not content.get("parts"):
            continue
        metadata = message.get("metadata")
        if not metadata:
            continue
        if message.get("author"):
            role = message.get("author").get("role")
        if metadata.get("is_visually_hidden_from_conversation"):
            continue
        else:
            parts = message.get("content").get("parts")
            model_slug = message.get("metadata").get("model_slug")
            if role == "user":
                markdown += "## <u>User prompt:</u>\n"
            elif model_slug:
                markdown += f"""## <u>{model_slug} response:</u>\n"""
            else:
                markdown += f"""## <u>{role} response:</u>\n"""
                markdown += "---\n"
            assert len(parts) == 1, parts
            markdown += f"""{parts[0]} \n\n"""
            markdown += "---  \n"
    global returns
    returns["markdown"] = markdown
    return markdown


def return_conversation_based_on_id(conversations, conversation_id):
    conversations = [conversation for conversation in conversations if conversation.get("conversation_id") == conversation_id]
    if conversations:
        return conversations[0]
    else:
        return None

def print_conv_titles(conversations):
    for conv_data in conversations:
        string = f"{conv_data.get("list_index")} - {conv_data.get("id")} - {conv_data.get("creation_iso_time")}- {conv_data.get("title")}"
        print(string)


def return_conv_by_dropdown_str(conversations, string):
    date, time = string.split()[:2]
    iso_time_str = " ".join([date, time])
    timestamp = int(datetime.fromisoformat(iso_time_str).timestamp())
    conversations = [conv for conv in conversations if int(conv.get("create_time")) == timestamp]
    if conversations and len(conversations) == 1:
        return conversations[0]
    else:
        return None


def render_to_output_chain(conversations, string):
    conversation = return_conv_by_dropdown_str(conversations, string)
    if not conversation:
        return
    markdown = conv_to_markdown(conversation)
    with output:
        output.clear_output()
        display(Markdown(markdown))
    return

### Widget initialization

In [3]:
case_sensitive = widgets.Checkbox(
    value=False, description="Case sensitive", disabled=False, indent=False
)


only_from_title = widgets.Checkbox(
    value=True, description="Search only from titles", disabled=False, indent=False
)


search_box = widgets.Text(
    value="",
    placeholder="Type something",
    description="Search string:",
    disabled=False,
    continuous_update=False,
    layout=widgets.Layout(width="400px"),
)


conv_selection = widgets.Select(
    options=["-"],
    value="-",
    # rows=25,
    description="Conversations:",
    disabled=False,
    layout=widgets.Layout(width="500px"),
)


search_box.observe(
    lambda x: conversation_search(x.get("new"), conversations, only_from_title.value, case_sensitive.value),
    names="value", type="change",
)


render_button = widgets.Button(
    description="Render selected",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Render selected conversation",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width="150px"),
)


render_button.on_click(
    lambda x: render_to_output_chain(conversations, conv_selection.value)
)


output = widgets.Output()
output.layout = widgets.Layout(width="960px", border="1px solid black", padding="50px")

### Import conversation history

In [4]:
config = ConfigParser()
config.read("config.ini")
conversation_path = config.get("conversations", "path")
with open(conversation_path, "r", encoding="utf-8") as file:
    conversations = json.load(file)

## Conversation search & rendering

In [5]:
items = [item for item in (search_box, only_from_title, case_sensitive, conv_selection, render_button, output)]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(1, None)"))

GridBox(children=(Text(value='', continuous_update=False, description='Search string:', layout=Layout(width='4…